In [1]:
import sys

sys.path.append(f"/home/npanj/personal_works/m5-forecasting-accuracy")

from src.utils.import_downcasting import import_downcasting
from src.utils.plotting import PdfFile
from utils import get_wmape, plotting_sales_forecast, get_wmape_custom_groupby
import os
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import copy

root = os.path.join(
    "/home/npanj/personal_works/m5-forecasting-accuracy/assets/data/data_CA_1"
)

In [2]:
data_ca1 = import_downcasting(os.path.join(root, "data_with_arima_resid.csv"))
score_ca1 = import_downcasting(os.path.join(root, "arima_model_score.csv"))

print("data ca1 list columns")
print(data_ca1.info(verbose=True))

print("--------------------------------------------------")
print("data ca1 score list columns")
print(score_ca1.info())

data_ca1 = data_ca1.set_index("date")
data_ca1["arima_residual"] = data_ca1.arima_residual.astype(np.float64)
data_ca1["event_name_1"] = (
    data_ca1["event_name_1"].cat.add_categories("none").fillna("none")
)
data_ca1["event_type_1"] = (
    data_ca1["event_type_1"].cat.add_categories("none").fillna("none")
)

data_ca1["event_name_1_group"] = data_ca1["event_name_1"].apply(lambda x: x if x == "none" else "special" )
data_ca1["event_type_1_group"] = data_ca1["event_type_1"].apply(lambda x: x if x == "none" else "special" )
data_ca1.info()

data ca1 list columns
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5918109 entries, 0 to 5918108
Data columns (total 25 columns):
 #   Column          Dtype         
---  ------          -----         
 0   date            datetime64[ns]
 1   id              category      
 2   item_id         category      
 3   dept_id         category      
 4   cat_id          category      
 5   store_id        category      
 6   state_id        category      
 7   date_code       category      
 8   sales           int16         
 9   wm_yr_wk        int16         
 10  weekday         category      
 11  wday            int8          
 12  month           int8          
 13  year            int16         
 14  d               category      
 15  event_name_1    category      
 16  event_type_1    category      
 17  event_name_2    category      
 18  event_type_2    category      
 19  snap_CA         int8          
 20  snap_TX         int8          
 21  snap_WI         int8          
 

- event
  May need to combine between event_1 and event_2

- snap
  Focus only on snap_CA (CA = Califonia, TX = texas, WI = witcostin)

In [3]:
##### Checking event_name_1 and event_type_1
check_event_name_type_1 = data_ca1[["event_name_1", "event_type_1", "sales"]].groupby(["event_name_1", "event_type_1"]).count()\
.reset_index()
check_event_name_type_1 = check_event_name_type_1[check_event_name_type_1.sales > 0]
check_event_name_type_1.sort_values(by="event_type_1", inplace=True)
# check_event_name_type_1


##### Checking event_name_2 and event_type_2
data_ca1.event_name_2.unique()

##### Checking snap_CA, snap_TX and snap_WI
check_snap = data_ca1[["snap_CA", "sales"]].groupby(["snap_CA"]).count()\
.reset_index()
check_snap.rename(columns={"sales": "num_obs"}, inplace=True)
# check_snap

##### Checking snap_CA, snap_TX and snap_WI
check_snap = data_ca1[["snap_CA", "sales"]].groupby(["snap_CA"]).count()\
.reset_index()
check_snap.rename(columns={"sales": "num_obs"}, inplace=True)
# check_snap

In [4]:
wmape = get_wmape(data_ca1, "arima_residual")
score_ca1 = pd.merge(
    left=score_ca1,
    right=wmape[["store_id", "item_id", "wmape"]],
    on=["store_id", "item_id"],
    how="left",
)

score_ca1.sort_values(by="wmape", ascending=False, inplace=True)
score_ca1["rank_pct_wmape"] = score_ca1["wmape"].rank(pct=True)
score_ca1 = score_ca1.reset_index(drop=True)

data_ca1["root_square_resid"] = data_ca1["arima_residual"].apply(
    lambda x: math.pow(x, 2)
)

In [5]:
item_id_cat_unq = data_ca1.groupby(["item_id", "cat_id"]).store_id.count().reset_index()
score_ca1 = pd.merge(
    left=score_ca1,
    right=item_id_cat_unq[["item_id", "cat_id"]],
    left_on=["item_id"],
    right_on=["item_id"],
    how="left",
)

score_ca1.info()
##########

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9147 entries, 0 to 9146
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   store_id        9147 non-null   category
 1   item_id         9147 non-null   category
 2   mse             9147 non-null   float16 
 3   mae             9147 non-null   float16 
 4   sse             9147 non-null   float32 
 5   wmape           9147 non-null   float64 
 6   rank_pct_wmape  9147 non-null   float64 
 7   cat_id          9147 non-null   category
dtypes: category(3), float16(2), float32(1), float64(2)
memory usage: 410.2 KB


In [6]:
# plotting_sales_forecast(data=data_ca1, score=score_ca1, file_name="plotting_store_ca1.pdf")
# plotting_sales_forecast(data=data_ca1, score=score_ca1, file_name="plotting_hobbies_ca1.pdf", product_cat="HOBBIES")
# plotting_sales_forecast(data=data_ca1, score=score_ca1, file_name="plotting_foods_ca1.pdf", product_cat="FOODS")
# plotting_sales_forecast(data=data_ca1, score=score_ca1, file_name="plotting_household_ca1.pdf",\
#                         product_cat="HOUSEHOLD")

# Analysis WMAPE on event_name_type1

- event_name_1 = "Christmas" has sales equal to 0 for all observastion
- Marjoriry of data belong to None type
- Only these following days that seem to have impact that cuase residual to be higher
        Thanksgiving
        NewYear
        Halloween
        LentStart
        LentWeek2
        ValentinesDay
        NBAFinalsStart
- grouping into two group comsprised of special and none. These feature seem to have little of impact 

In [7]:
wmape_by_event_name_1 = get_wmape_custom_groupby(data_ca1, "arima_residual", ["event_name_1"])
wmape_by_event_name_1.sort_values(by="wmape", ascending=False, inplace=True)
wmape_by_event_name_1["pct_obs"] = wmape_by_event_name_1["num_obs"]/wmape_by_event_name_1.num_obs.sum()
wmape_by_event_name_1.to_csv("wmape_analysis/wmape_by_event_name_1.csv")

wmape_event_name_1_grp = get_wmape_custom_groupby(data_ca1, "arima_residual", ["event_name_1_group"])
wmape_event_name_1_grp.to_csv("wmape_analysis/wmape_event_name_1_group.csv")

# Analysis WMAPE on event_type_1

- Only these following days that seem to have impact that cuase residual to be higher
        National (Highest Different)
        Cultural
- Grouping to special and non special seem to have none impact

In [8]:
wmape_by_event_type_1 = get_wmape_custom_groupby(data_ca1, "arima_residual", ["event_type_1"])
wmape_by_event_type_1.sort_values(by="wmape", ascending=False, inplace=True)
wmape_by_event_type_1["pct_obs"] = wmape_by_event_type_1["num_obs"]/wmape_by_event_type_1.num_obs.sum()
wmape_by_event_type_1.to_csv("wmape_analysis/wmape_by_event_type_1.csv")

wmape_by_event_type_1_group = get_wmape_custom_groupby(data_ca1, "arima_residual", ["event_type_1_group"])
wmape_by_event_type_1_group.to_csv("wmape_analysis/wmape_by_event_type_1_group.csv")

# Analysis WMAPE on snap_CA

- WMAPE seems indifferent between having a snap and not having a snap

In [14]:
wmape_snap = get_wmape_custom_groupby(data_ca1, "arima_residual", ["snap_CA"])
wmape_snap.sort_values(by="wmape", ascending=False, inplace=True)
wmape_snap["pct_obs"] = wmape_snap["num_obs"]/wmape_snap.num_obs.sum()
wmape_snap.to_csv("wmape_analysis/wmape_snap.csv")

# Analysis WMAPE on DEPARTMENT

In [ ]:
wmape_snap = get_wmape_custom_groupby(data_ca1, "arima_residual", ["snap_CA"])
wmape_snap.sort_values(by="wmape", ascending=False, inplace=True)
wmape_snap["pct_obs"] = wmape_snap["num_obs"]/wmape_snap.num_obs.sum()
wmape_snap.to_csv("wmape_analysis/wmape_snap.csv")

# Analysis residual from ARIMA model in the group last 20 percentile

In [10]:
resid_score_worst_20pct = score_ca1[score_ca1.rank_pct_wmape > 0.8]
item_resid_worst_20pct = list(resid_score_worst_20pct.item_id.unique())

data_worst_20pct = data_ca1[data_ca1.item_id.isin(item_resid_worst_20pct)]
data_worst_20pct.to_csv(os.path.join(root, "data_arima_worst_30pct.csv"))

data_worst_20pct = import_downcasting(os.path.join(root, "data_arima_worst_30pct.csv"))

<span style="color:blue">Explore event_name_1 and event_type_1</span>

Finding from rank in event_name_1 that "christmas" event tend to have the most impact on residual

In [11]:
event_name_1 = (
    data_worst_20pct.groupby(["event_name_1"]).root_square_resid.mean().reset_index()
)
event_name_1.rename(columns={"root_square_resid": "mean_rss"}, inplace=True)
event_name_1.sort_values(by="mean_rss", ascending=False, inplace=True)


event_type_1 = (
    data_worst_20pct.groupby(["event_type_1"]).root_square_resid.mean().reset_index()
)
event_type_1.rename(columns={"root_square_resid": "mean_rss"}, inplace=True)
event_type_1.sort_values(by="mean_rss", ascending=False, inplace=True)

In [12]:
event_name_1.head(10)

,event_name_1,mean_rss
5,Eid al-Fitr,0.332275
15,Mother's day,0.300049
29,VeteransDay,0.298584
30,none,0.291260
20,OrthodoxEaster,0.288574
10,LaborDay,0.258789
6,EidAlAdha,0.256348
16,NBAFinalsEnd,0.244995
23,Purim End,0.242798
25,StPatricksDay,0.233276


In [13]:
<span style="color:blue">Explore event_name_1 and event_type_1</span>

SyntaxError: invalid syntax (483449040.py, line 1)